In [32]:
from bs4 import BeautifulSoup as bs
import urllib2
import re
import csv
from collections import OrderedDict
import cookielib
import time

In [33]:
# start = 'http://archiveofourown.org/tags/Sherlock%20(TV)/works'
# outfile = '/Users/jingy/Desktop/ao3/data/ao3_work_sherlock_all_2.csv'
# max_page = 3854

start = 'http://archiveofourown.org/tags/Curufin*s*Finrod/works'
outfile = '/Users/jingy/Desktop/finrod_curufin.csv'
max_page = 4

In [34]:
cookie_file = '/Users/jingy/Desktop/ao3/code/cookie'

In [35]:
def save_cookie(cookie_file):
    cookie = cookielib.MozillaCookieJar(cookie_file)
    opener = urllib2.build_opener(urllib2.HTTPCookieProcessor(cookie))
    response = opener.open('http://archiveofourown.org/works/5051548?view_adult=true')
    cookie.save(ignore_discard=True, ignore_expires=True)

In [36]:
def load_cookie(cookie_file):
    cookie = cookielib.MozillaCookieJar()
    cookie.load(cookie_file, ignore_discard=True, ignore_expires=True)
    return cookie

In [37]:
# save_cookie(cookie_file)
cookie = load_cookie(cookie_file)

In [38]:
def find_page(base_url, page_number):
    #go to any page number.
    return base_url+'?page=' +str(page_number)

In [39]:
def find_works(page):
    #Find all works from a works list page.
    works_page = bs(urllib2.urlopen(page))
    links = []
    for link in works_page.find_all('a'):
        url = link.get('href')
        url_s = [i for i in url.split('/') if i != '']
        if 'work' in url and len(url_s) == 2 and str(url_s[1]).isdigit():
                links.append('http://archiveofourown.org'+link.get('href'))
    return links

In [40]:
def show_full_contents(url):
    #go through adult contents filtering.
    base = bs(urllib2.urlopen(url))
    full_url = url
    for link in base.find_all('a'):
        if 'Proceed' in link.text:
            full_url = url +'?view_adult=true'
    return full_url

In [41]:
def get_next_chapter(url):
    #for multi chapter works, get next chapter link.
    req = urllib2.Request(url)
    opener = urllib2.build_opener(urllib2.HTTPCookieProcessor(cookie))
    page = bs(opener.open(req))
    next_chapter = ''
    for link in page.find_all('a'):
        if 'Next Chapter' in link.text:
            next_chapter = ('http://archiveofourown.org' + link.get('href'))
    return next_chapter

In [42]:
def get_download_link(url):
    #may use the download page alternatively, where the html page is cleaner. But this misses some features, so not use
    #at the moment.
    req = urllib2.Request(url)
    opener = urllib2.build_opener(urllib2.HTTPCookieProcessor(cookie))
    page = bs(opener.open(req))
    for link in page.find_all('a'):
        if 'HTML' in link.text:
            download_link = 'http://archiveofourown.org' + link.get('href')
    return download_link

In [43]:
def get_contents(url):
#     get work metadata and contents from the work page.
#     print 'Reading url:', url
    try:
        req = urllib2.Request(url)
        opener = urllib2.build_opener(urllib2.HTTPCookieProcessor(cookie))
        page = bs(opener.open(req))
        contents = str(page.body.text.encode('utf-8')).replace('\n','A')
    except:
        print 'Unable to read from this url'
        contents = ''
        with open('/Users/jingy/Desktop/problematic_url.csv', 'a') as g:
            g.write(url)
    return url, contents

In [44]:
def write_header(outfile):
    f = open(outfile, 'a')
    writer = csv.writer(f, delimiter=',')
    keys = ['Additional_Tags', 'Archive_Warnings', 'Author', 'Bookmarks', 'Category', 'Chapters', 'Characters',\
             'Comments', 'CompleteDate', 'Fandoms', 'Hits', 'Kudos', 'Language', 'Notes', 'PublishDate', 'Rating',\
             'Relationship', 'Summary', 'Text', 'Title', 'Words']
    writer.writerow(keys)
    f.close()

In [45]:
def write_work_content(work_dict,outfile):
    #write work metadata and contents as values of a sorted dictionary.
    f = open(outfile, 'a')
    writer = csv.writer(f, delimiter=',')
    try:
        writer.writerow(OrderedDict(sorted(work_dict.items())).values())
    except:
        pass
    f.close()

In [46]:
#creates dictionary for information in a single work.
def create_work_dict(url, contents):
#     get work metadata and contents into a dictionary.

#     print 'Getting work information from:', url
    try:
        work = {}

        rating = re.findall('Rating:(.*?)<br />',contents) 
        warning = re.findall('Warnings:(.*?)<br />',contents)
        fandom = re.findall('Fandoms:A          AAA(.*?)AAAA|Fandom:A          AAA(.*?)AAAA',contents) 
        category = re.findall('Category:A          AAA(.*?)AAAA',contents)
        relationship = re.findall('Relationships:(.*?)<br />',contents)
        characters = re.findall('Characters:(.*?)<br />',contents)
        additional = re.findall('Additional Tags:(.*?)<br />',contents) 
        language = re.findall('Language:A      AA(.*?)A      A',contents)
        author = re.findall('A    AA(.*?)AAA',contents)
        text = re.findall('Work Text:(.*?)AAAAAAAA|Chapter TextA(.*?)AAAAA|Chapter TextAAAAAA(.*?)',contents)
        text = [i for i in text[0] if i != ''] if text != [] else []
        title = re.findall('AAAAAAAA      (.*?)A    AA',contents)
        summary = re.findall('>Summary: <p>(.*?)</p>',contents)
        notes = re.findall('Notes:AA(.*?)AA',contents)
        publishdate = re.findall('Published:([0-9]*-[0-9]*-[0-9]*)',contents)
        completedate = re.findall('Completed:([0-9]*-[0-9]*-[0-9]*)',contents)
        words = re.findall('Words:([0-9]*)',contents)
        chapters = re.findall('Chapters:([0-9]*/[0-9]*)',contents)
        comments = re.findall('Comments:([0-9]*)',contents)
        kudos = re.findall('Kudos:([0-9]*)',contents)
        bookmarks = re.findall('Bookmarks:([0-9]*)',contents)
        hits = re.findall('Hits:([0-9]*)',contents)       
        
        work['Rating'] = rating[0] if rating != [] else ''
        work['Archive_Warnings'] = warning[0] if warning != [] else ''
        work['Fandoms'] = [i for i in fandom[0] if i != ''][0] if fandom != [] else ''
        work['Category'] = category[0] if category != [] else ''
        work['Relationship'] = relationship[0] if relationship != [] else '' 
        work['Characters'] = characters[0] if characters != [] else ''
        work['Additional_Tags'] = additional[0] if additional != [] else ''
        work['Language'] = language[0] if language != [] else ''
        work['Author'] = author[0] if author != [] else ''
        work['Text']= text[0] if text != [] else ''
        work['Title']  = title[0] if title != [] else ''
        work['Summary'] = summary[0] if summary != [] else ''
        work['Notes'] = notes[0] if notes != [] else ''
        work['Comments'] = comments[0] if comments != [] else ''
        work['PublishDate'] = publishdate[0] if publishdate != [] else ''
        work['CompleteDate'] = completedate[0] if completedate != [] else ''
        work['Words'] = words[0] if words != [] else ''
        work['Chapters'] = chapters[0] if chapters != [] else ''
        work['Comments'] = comments[0] if comments != [] else ''
        work['Kudos'] = kudos[0] if kudos != [] else ''
        work['Bookmarks'] = bookmarks[0] if bookmarks != [] else ''
        work['Hits'] = hits[0] if hits != [] else ''
    
#     print 'Finished with:', url
    except:
        print 'Something went wrong.'
    return work

In [47]:
def read_single_work(url):
    url_full = show_full_contents(url)
    u, c = get_contents(url_full)
    work = create_work_dict(u, c)
    write_work_content(work,outfile)

In [48]:
def get_chapters(url):
    url_full = show_full_contents(url)
    chapters_list = []
    next_url = get_next_chapter(url_full)
    while next_url != '':
        chapters_list.append(next_url)
        next_url = get_next_chapter(next_url)
    return chapters_list

In [49]:
# get_next_chapter('http://archiveofourown.org/works/3078407?view_adult=true')

In [50]:
# s = get_contents('http://archiveofourown.org/works/5051548?view_adult=true')

In [51]:
# d = create_work_dict('u',str(s))

In [52]:
# d = create_work_dict('http://archiveofourown.org/works/5205566',str(c))

In [53]:
# c = bs(urllib2.urlopen('http://archiveofourown.org/works/3078407/chapters/6678515')).body.text

In [54]:
# get_chapters('http://archiveofourown.org/works/3078407?view_adult=true')

In [55]:
#main loop
write_header(outfile)
start_time = time.clock()
count = 0

for i in range(1, max_page+1):
    page = find_page(start, i)
    worklist = find_works(page)
    for w in worklist:
        ch_list = get_chapters(w)
        if ch_list != []:
            read_single_work(w)
            for ch in ch_list:
                read_single_work(ch)
        else:
            read_single_work(w)
        count += 1
        
#     if count%20 == 0:
    print 'crawling page:', i
#         print 'reading work:', w

#done:-964

print 'Saved %s works from %s pages of tag %s in %s seconds .' %(count, i, 'Finrod/Curufin', str(time.clock() - start_time))        

crawling page: 1
crawling page: 2
crawling page: 3
crawling page: 4
Saved 70 works from 4 pages of tag Finrod/Curufin in 91.858123 seconds .
